In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import scipy.stats as stats 
%matplotlib inline

In [3]:
nypd = pd.read_csv('../data/nypd.csv')
nypd['Date'] = pd.to_datetime(nypd['OCCUR_DATE'])
nypd = nypd.drop('OCCUR_DATE', axis=1)

In [65]:
nypd['Month'] = nypd.Date.dt.month 


In [67]:
nypd['Year'] = nypd.Date.dt.year

In [68]:
nypd.head()

,INCIDENT_KEY,OCCUR_TIME,BORO,PRECINCT,JURISDICTION_CODE,LOCATION_DESC,STATISTICAL_MURDER_FLAG,PERP_AGE_GROUP,PERP_SEX,PERP_RACE,...,VIC_SEX,VIC_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat,Date,Month,Year
0,201575314,22:10:00,QUEENS,103,0.0,NaN,False,NaN,NaN,NaN,...,M,BLACK,1037451,193561,40.697805,-73.808141,POINT (-73.80814071699996 40.697805308000056),2019-08-23,8,2019
1,205748546,15:54:00,BRONX,40,0.0,NaN,False,<18,M,BLACK,...,F,BLACK,1006789,237559,40.818700,-73.918571,POINT (-73.91857061799993 40.81869973000005),2019-11-27,11,2019
2,193118596,19:40:00,MANHATTAN,23,0.0,NaN,False,18-24,M,WHITE HISPANIC,...,M,BLACK HISPANIC,999347,227795,40.791916,-73.945480,POINT (-73.94547965999999 40.791916091000076),2019-02-02,2,2019
3,204192600,00:52:00,STATEN ISLAND,121,0.0,PVT HOUSE,True,25-44,M,BLACK,...,F,BLACK,938149,171781,40.638064,-74.166108,POINT (-74.16610830199996 40.63806398200006),2019-10-24,10,2019
4,201483468,18:03:00,BRONX,46,0.0,NaN,False,25-44,M,BLACK HISPANIC,...,M,BLACK,1008224,250621,40.854547,-73.913339,POINT (-73.91333944399999 40.85454734900003),2019-08-22,8,2019


In [69]:
#dividing dataframes into boroughs
bronx = nypd.loc[nypd.BORO == 'BRONX']
brooklyn = nypd.loc[nypd.BORO == 'BROOKLYN']
queens = nypd.loc[nypd.BORO == 'QUEENS']
staten = nypd.loc[nypd.BORO == 'STATEN ISLAND']
manhattan = nypd.loc[nypd.BORO == 'MANHATTAN']

In [71]:
#dividing borough dataframes into 
bx_k = bronx.loc[bronx.STATISTICAL_MURDER_FLAG == True]
brooklyn_k = brooklyn.loc[brooklyn.STATISTICAL_MURDER_FLAG == True]
queens_k = queens.loc[queens.STATISTICAL_MURDER_FLAG == True]
staten_k = staten.loc[staten.STATISTICAL_MURDER_FLAG == True]
manhattan_k = manhattan.loc[manhattan.STATISTICAL_MURDER_FLAG == True]



In [79]:
#assigning total murders by borough
bx_killed, bk_killed, queens_killed, staten_killed, manhattan_killed = bx_k.count()['INCIDENT_KEY'], brooklyn_k.count()['INCIDENT_KEY'], queens_k.count()['INCIDENT_KEY'], staten_k.count()['INCIDENT_KEY'], manhattan_k.count()['INCIDENT_KEY']

In [84]:
for name, group in nypd.groupby('LOCATION_DESC'):
    print(name)
    print(group)

ATM
       INCIDENT_KEY OCCUR_TIME   BORO  PRECINCT  JURISDICTION_CODE  \
10129     188884200   12:21:00  BRONX        43                2.0   

      LOCATION_DESC  STATISTICAL_MURDER_FLAG PERP_AGE_GROUP PERP_SEX  \
10129           ATM                    False            NaN      NaN   

      PERP_RACE  ... VIC_SEX VIC_RACE X_COORD_CD Y_COORD_CD   Latitude  \
10129       NaN  ...       M    BLACK  1,021,375    236,033  40.814462   

       Longitude                           Lon_Lat       Date Month  Year  
10129 -73.865882  POINT (-73.86588157 40.81446204) 2018-10-17    10  2018  

[1 rows x 21 columns]
BANK
       INCIDENT_KEY OCCUR_TIME   BORO  PRECINCT  JURISDICTION_CODE  \
14767      87339334    8:35:00  BRONX        41                0.0   

      LOCATION_DESC  STATISTICAL_MURDER_FLAG PERP_AGE_GROUP PERP_SEX  \
14767          BANK                    False            NaN      NaN   

      PERP_RACE  ... VIC_SEX        VIC_RACE X_COORD_CD Y_COORD_CD  Latitude  \
14767       NaN

[11 rows x 21 columns]
DRY CLEANER/LAUNDRY
       INCIDENT_KEY OCCUR_TIME           BORO  PRECINCT  JURISDICTION_CODE  \
1034       80758195   22:28:00       BROOKLYN        73                0.0   
2284       51787776    7:30:00       BROOKLYN        75                0.0   
3176       10619049    2:00:00      MANHATTAN        25                0.0   
3177       62242976   22:17:00       BROOKLYN        67                0.0   
5577       79238544   15:11:00         QUEENS       113                0.0   
5803       73137862   12:34:00  STATEN ISLAND       120                0.0   
7101      136431300   14:00:00       BROOKLYN        88                0.0   
7746       87670468   22:05:00         QUEENS       114                0.0   
7826       80581458    5:50:00         QUEENS       101                0.0   
8224      137591691   21:35:00       BROOKLYN        79                0.0   
9031       45970311    3:13:00         QUEENS       113                0.0   
9337       75948843  

[24 rows x 21 columns]
JEWELRY STORE
       INCIDENT_KEY OCCUR_TIME       BORO  PRECINCT  JURISDICTION_CODE  \
4825       69379175   14:00:00   BROOKLYN        62                0.0   
4858       65012637   16:53:00   BROOKLYN        73                0.0   
4906      163631340   12:13:00      BRONX        50                0.0   
6083       10619080   11:15:00   BROOKLYN        90                0.0   
6590       10619080   11:15:00   BROOKLYN        90                0.0   
7658       70964821   12:25:00  MANHATTAN        19                0.0   
8796       54059281   14:21:00  MANHATTAN        18                0.0   
13920      69379175   14:00:00   BROOKLYN        62                0.0   
17222      79969482   12:49:00   BROOKLYN        94                0.0   
18799      10187498   19:49:00     QUEENS       102                0.0   
20147      92747225   11:15:00   BROOKLYN        72                0.0   
21040      69379175   14:00:00   BROOKLYN        62                0.0   



       INCIDENT_KEY OCCUR_TIME   BORO  PRECINCT  JURISDICTION_CODE  \
12070      10240239   22:37:00  BRONX        52                0.0   

      LOCATION_DESC  STATISTICAL_MURDER_FLAG PERP_AGE_GROUP PERP_SEX  \
12070        SCHOOL                     True            NaN        U   

      PERP_RACE  ... VIC_SEX VIC_RACE X_COORD_CD Y_COORD_CD   Latitude  \
12070   UNKNOWN  ...       M    BLACK    1017542     255919  40.869059   

       Longitude                                       Lon_Lat       Date  \
12070  -73.87963  POINT (-73.87963014799993 40.86905853200005) 2006-01-23   

      Month  Year  
12070     1  2006  

[1 rows x 21 columns]
SHOE STORE
       INCIDENT_KEY OCCUR_TIME      BORO  PRECINCT  JURISDICTION_CODE  \
673       204971620   19:30:00  BROOKLYN        71                0.0   
1644       84854142   19:31:00  BROOKLYN        71                0.0   
2503       51110049   19:29:00     BRONX        44                0.0   
3634       84854142   19:31:00  BROOKLYN    